<a href="https://colab.research.google.com/github/hibahassan1/COVID19-Tweets-Sentiment-Analysis/blob/main/Sentiment_Analysis_Training_and_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
twitter_data = pd.read_csv('/content/drive/MyDrive/covid19_tweets.csv')

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(content):
    content = content.lower()
    words = re.findall(r'\b\w+\b', content)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    preprocessed_content = ' '.join(lemmatized_words)
    return preprocessed_content


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
twitter_data['preprocessed_text'] = twitter_data['Tweets'].apply(preprocess_text)

In [ ]:
twitter_data.head()

,Tweets,ID,Date,Likes,Retweets,Sentiment,preprocessed_text
0,Me to my Will you with me for the rest of our...,1.320000e+18,11/10/2020 11:20,2,1,1,rest life
1,"Yes Offcourse, most of the tested people here ...",1.320000e+18,11/10/2020 10:45,1,0,2,yes offcourse tested people foreigner approx 7...
2,Tune in to Dubai Eye tomorrow at 4:00 pm. Ludm...,1.320000e+18,11/10/2020 6:32,0,0,1,tune dubai eye tomorrow 4 00 pm ludmila yamalo...
3,Are you sure about that sir Students died afte...,1.320000e+18,11/10/2020 6:23,1,0,0,sure sir student died contracting corona atten...
4,I hope Delhi government will help his family,1.320000e+18,11/10/2020 5:53,0,0,2,hope delhi government help family


Applying Cross Validation to improve Accuracy


In [ ]:
#Training the SVM
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

# Load your dataset and preprocess it
# Define your features (X) and target (y)
X = twitter_data['preprocessed_text']
y = twitter_data['Sentiment']

# Convert text data into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Apply SMOTE to upsample the minority classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Training the SVM
svm_model = SVC(kernel='rbf', C=10.0, gamma=1.0)

# Perform cross-validation
cv_scores = cross_val_score(svm_model, X_train, y_train, cv=5)

# Fit the model on the entire training set
svm_model.fit(X_train, y_train)

# Predict labels for the test set
y_pred = svm_model.predict(X_test)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

# Save the trained model
# joblib.dump(svm_model, 'svm_model.pkl')



Cross-validation scores: [0.76446669 0.74624531 0.74561952 0.75938673 0.75125156]
Mean CV accuracy: 0.7533939632861376
Accuracy: 0.7805305305305306
              precision    recall  f1-score   support

           0       0.82      0.79      0.80      1355
           1       0.70      0.73      0.71      1320
           2       0.84      0.82      0.83      1321

    accuracy                           0.78      3996
   macro avg       0.78      0.78      0.78      3996
weighted avg       0.78      0.78      0.78      3996



In [ ]:
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

In [ ]:
#Training LR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import joblib
import numpy as np

# Define your features (X) and target (y)
X = twitter_data['preprocessed_text']
y = twitter_data['Sentiment']

# Convert text data into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Apply SMOTE to upsample the minority classes
smote = SMOTE(random_state=2)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Training the Logistic Regression
lr_model = LogisticRegression(C=3.329770563201687, penalty='l2', solver='saga', max_iter=500)

# Perform cross-validation
cv_scores = cross_val_score(lr_model, X_train, y_train, cv=5)

# Fit the model on the entire training set
lr_model.fit(X_train, y_train)

# Predict labels for the test set
y_pred = lr_model.predict(X_test)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

# Save the trained model
# joblib.dump(lr_model, 'lr_model.pkl')


Cross-validation scores: [0.71254301 0.6971214  0.69211514 0.70588235 0.68742178]
Mean CV accuracy: 0.699016736983162
Accuracy: 0.7112112112112112
              precision    recall  f1-score   support

           0       0.75      0.75      0.75      1355
           1       0.64      0.62      0.63      1320
           2       0.74      0.77      0.75      1321

    accuracy                           0.71      3996
   macro avg       0.71      0.71      0.71      3996
weighted avg       0.71      0.71      0.71      3996



In [ ]:
#Training Adaboost
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
import joblib
import pandas as pd

# Load your dataset and preprocess it
# Define your features (X) and target (y)
X = twitter_data['preprocessed_text']
y = twitter_data['Sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Apply SMOTE to upsample the minority classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Train the AdaBoostClassifier
adaboost_model = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(adaboost_model, X_resampled, y_resampled, cv=5)

# Fit the model on the entire resampled dataset
adaboost_model.fit(X_resampled, y_resampled)

# Predict labels for the test set
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred = adaboost_model.predict(X_test_tfidf)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

# Save the trained model
# joblib.dump(adaboost_model, 'adaboost_model.pkl')


Cross-validation scores: [0.51326326 0.55005005 0.62377972 0.67459324 0.62553191]
Mean CV accuracy: 0.597443638882938
Accuracy: 0.6457611093602269
              precision    recall  f1-score   support

           0       0.73      0.64      0.69      1062
           1       0.60      0.68      0.64      1275
           2       0.63      0.59      0.61       836

    accuracy                           0.65      3173
   macro avg       0.65      0.64      0.64      3173
weighted avg       0.65      0.65      0.65      3173



In [ ]:
#Training random forest
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Define your features (X) and target (y)
X = twitter_data['preprocessed_text']
y = twitter_data['Sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Apply SMOTE to upsample the minority classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Initialize RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=200,
                                   max_depth=20,
                                   min_samples_split=2,
                                   min_samples_leaf=1,
                                   max_features='sqrt',
                                   random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(rf_model, X_resampled, y_resampled, cv=5)

# Fit the model on the entire resampled dataset
rf_model.fit(X_resampled, y_resampled)

# Make predictions
X_test_tfidf = tfidf_vectorizer.transform(X_test)
y_pred = rf_model.predict(X_test_tfidf)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

# Calculate and print accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print(classification_report(y_test, y_pred))

# Save the trained model
# joblib.dump(rf_model, 'rf_model.pkl')


Cross-validation scores: [0.47522523 0.56656657 0.60625782 0.66382979 0.60250313]
Mean CV accuracy: 0.582876506042964
Accuracy: 0.7046958714150646
              precision    recall  f1-score   support

           0       0.80      0.62      0.70      1062
           1       0.65      0.79      0.71      1275
           2       0.71      0.68      0.69       836

    accuracy                           0.70      3173
   macro avg       0.72      0.70      0.70      3173
weighted avg       0.72      0.70      0.70      3173



In [ ]:
# Saving and Loading the SVM model
joblib.dump(svm_model, 'svm_model.pkl')
loaded_model = joblib.load('svm_model.pkl')

In [ ]:
#Checking Logistic Regression
loaded_model = joblib.load('lr_model.pkl')

In [ ]:
#Checking Random Forest
loaded_model = joblib.load('rf_model.pkl')

In [ ]:
#Checking Adaboost classifier
loaded_model = joblib.load('adaboost_model.pkl')

In [ ]:
def predict_and_print_sentiment(X_new):
    # Initialize an empty list to store preprocessed tweets
    preprocessed_X_new = []

    # Preprocess each tweet in the list
    for tweet in X_new:
        preprocessed_tweet = preprocess_text(tweet)
        preprocessed_X_new.append(preprocessed_tweet)

    # Convert preprocessed tweets into vectors
    X_new_vectors = tfidf_vectorizer.transform(preprocessed_X_new)

    # Make predictions using the loaded model
    predictions = loaded_model.predict(X_new_vectors)

    # Print predictions
    for prediction in predictions:
        if prediction == 0:
            print("Negative")
        elif prediction == 1:
            print("Neutral")
        else:
            print("Positive")


In [ ]:
X_new = ["What will happen to the iPhone 2020 with the Corona tragedy?"]

predict_and_print_sentiment(X_new)

Neutral


In [ ]:
X_new = ["I'm so grateful for the scientists and healthcare workers who are working tirelessly to fight COVID-19. Together, we will overcome this challenge. #gratitude #COVID19"]
# Initialize an empty list to store preprocessed tweets
preprocessed_X_new = []

# Preprocess each tweet in the list
for tweet in X_new:
    preprocessed_tweet = preprocess_text(tweet)
    preprocessed_X_new.append(preprocessed_tweet)

# Convert preprocessed tweets into TF-IDF vectors
X_new_tfidf = tfidf_vectorizer.transform(preprocessed_X_new)

# Make predictions using the loaded model
prediction = loaded_model.predict(X_new_tfidf)

print(prediction)

[2]


In [ ]:
X_new = ["The latest statistics on COVID-19 cases are out. It's important to stay informed and continue following safety guidelines to protect ourselves and others. #COVID19"]
# Initialize an empty list to store preprocessed tweets
preprocessed_X_new = []

# Preprocess each tweet in the list
for tweet in X_new:
    preprocessed_tweet = preprocess_text(tweet)
    preprocessed_X_new.append(preprocessed_tweet)

# Convert preprocessed tweets into TF-IDF vectors
X_new_tfidf = tfidf_vectorizer.transform(preprocessed_X_new)

# Make predictions using the loaded model
prediction = loaded_model.predict(X_new_tfidf)

print(prediction)

[1]
